In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc
plt.rcParams["figure.figsize"] = [12,12]
#If you have problems with latex at matplotlib just comment next two lines, this might help
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
def fix_scaling(ax=None):
    if not ax:
        xlim = plt.xlim()
        ylim = plt.ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            plt.ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            plt.xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))
    else:
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        d1 = xlim[1] - xlim[0]
        d2 = ylim[1] - ylim[0]
        if d1 > d2:
            ax.set_ylim((ylim[0] - (d1-d2) / 2, ylim[1] + (d1-d2) / 2))
        else:
            ax.set_xlim((xlim[0] + (d1-d2) / 2, xlim[1] - (d1-d2) / 2))

In [3]:
def func(x):
    return np.exp(x[0] + 3 * x[1]) + np.exp(x[0] - 3 * x[1])  + np.exp(-x[0]) 
def f_grad(x):
    return np.array([np.exp(x[0] + 3 * x[1]) + np.exp(x[0] - 3 * x[1]) - np.exp(-x[0]),
                     3 * np.exp(x[0] + 3 * x[1]) - 3 * np.exp(x[0] - 3 * x[1])])

In [4]:
def get_Lipschitz():
    delta = 0.05
    x = np.arange(-1, 1, delta)
    y = np.arange(-1, 1, delta)
    X, Y = np.meshgrid(x, y)
    L = np.linalg.norm(f_grad([1, 1])) / np.linalg.norm([1, 1])
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            x1 = np.array([X[i][j], Y[i][j]])
            for k in range(X.shape[0]):
                for t in range(X.shape[1]):
                    x2 = np.array([X[k][t], Y[k][t]])
                    if np.linalg.norm(x1 - x2) != 0:
                        L = max(L, np.linalg.norm(f_grad(x1) - f_grad(x2)) / np.linalg.norm(x1 - x2))
    return L

In [5]:
Lip = get_Lipschitz()

In [46]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
argmin = np.array([-np.log(2)/2, 0])
def animate_trajectory(trajes, colors):
    m = len(trajes)
    fig, ax = plt.subplots()
    n = 20
    def step(t):
            ax.cla()
            ax.plot([argmin[0]], [argmin[1]], 'o', color='green')
            #Level contours
            delta = 0.025
            x = np.arange(-3, 3, delta)
            y = np.arange(-3, 3, delta)
            X, Y = np.meshgrid(x, y)
            Z = np.zeros_like(X)
        #print(X.shape, Y.shape)
            for i in range(X.shape[0]):
                for j in range(X.shape[1]):
                    Z[i][j] = func([X[i][j], Y[i][j]])
            CS = ax.contour(X, Y, Z, [2, 6, 20], colors=['blue', 'purple', 'red'])
            for i in range(0,m):
                traj = trajes[i]
                ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], color=colors[i])
                ax.plot([u[0] for u in traj[:t]], [u[1] for u in traj[:t]], 'o', color=colors[i])
        
                fix_scaling(ax)
            ax.axis('off')

    return FuncAnimation(fig, step,
                     frames=range(n), interval=600)

In [47]:
alpha = 1.5 / Lip
traj_opt_step = []
x_start = np.array([1, 1])
traj_opt_step.append(x_start.copy())
cur_x = x_start.copy()
for i in range(20):
    cur_x = cur_x - alpha * f_grad(cur_x)
    traj_opt_step.append(cur_x.copy())

In [49]:
#Nesterov
traj_nesterov = []
x_start = np.array([1, 1])
traj_nesterov.append(x_start.copy())
cur_x = x_start.copy()
cur_y = x_start.copy()
alpha = 0.5

for i in range(20):
    alpha_prev = alpha
    x_prev = cur_x
    cur_x = cur_y - 1 / Lip * f_grad(cur_y)
    alpha = 0.5 * (-alpha ** 2 + alpha * np.sqrt(alpha ** 2 + 4))
    beta = (alpha_prev * (1 - alpha_prev)) / (alpha_prev ** 2 + alpha)
    cur_y = cur_x + beta * (cur_x - x_prev)
    traj_nesterov.append(cur_x.copy())
    
#print(traj)
base_animation = animate_trajectory([traj_opt_step, traj_nesterov], 
                                   ['black', 'deeppink'])
HTML(base_animation.to_html5_video())  